In [1]:
%pylab inline
import pandas as pd
import os
from threading import Thread
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import regex
import nltk
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from threading import Thread
import math
import unidecode
from params import *
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump, load
import time
import numpy as np
from oued_timing import Timer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from pathlib import Path
from numba import jit, njit
import numba
#nltk.downlod('all')


Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("./result.csv")

In [3]:
%time array = np.array(df["id"], dtype=np.int64)

CPU times: user 5.46 ms, sys: 7.81 ms, total: 13.3 ms
Wall time: 10.7 ms


In [4]:
"""
 had la class fiha les method ga3 ta3 text preprocessing
"""
class Preprocessor:
    def __init__(self):
        self.lemmatizer = FrenchLefffLemmatizer()
        self.french_stopwords = set(stopwords.words('french'))
        self.stemmer = nltk.stem.snowball.FrenchStemmer()

    def get_int_array(self, str_table):
        """
        @param str_table: str => un tableau d'entier 
        @return [int]
        """
        ok = False
        value = 0
        ans = []
        for c in str_table:
            if not c.isnumeric():
                if ok:
                    ans.append(value)
                    value = 0
                ok = False
            else:
                ok = True
                value *= 10
                value += int(c)
        assert ans
        return ans
    def get_int_array_min(self, str_table):
        return min(self.get_int_array(str_table))
    def filter_stopfr(self, text):
        return [token for token in text if token not in self.french_stopwords]
    def clean_txt(self, text):
        text = str(text)
        text = text.lower()
        text = re.sub("""[\.\!\"\s\?\-\,\'@&^$`]+""", " ",text)
        text = re.sub("""(\W|\d)+""", " ",text)
        text = regex.sub(u'[^\p{Latin}]', u' ', text)
        words = self.filtre_stopfr(word_tokenize(text, language="french") )
        words = [self.lemmatizer.lemmatize(unidecode.unidecode(w)) for w in words]
        words = [w for w in words if 3 < len(w) and len(w) < 13]
        #words = [self.stemmer.stem(w) for w in words]
        return " ".join(words)



In [43]:



def all(src_file_path, dest_file_path): 
    df = pd.read_csv(src_file_path); 
    pre = Preprocessor()
    def handle(df, start, end):
        print(f"start {start} {end}")
        end = min(end, df.shape[0])
        df.loc[start:end, "title"] = df.loc[start:end,"title"].apply(pre.clean_txt)
        df.loc[start:end, "description"] = df.loc[start:end,"description"].apply(pre.clean_txt)
        print(f"end {start} {end}")
    
    rows = df.shape[0]
    step = math.ceil(rows / NBR_THREADS)
    threads = []
    df["root"] = df["categories"].apply(pre.get_int_array_min)
    for i in range(0, rows, step):
        threads.append(Thread(target=handle, args=(df, i, min(rows, i + step))))
        threads[-1].start()
    for t in threads:
        t.join()
    df["title"] = df["title"].fillna(" ")
    df["description"] = df["description"].fillna(" ")
    df["text"] = df["title"] + " " + df["description"]
    df["text"] = df["text"].fillna(" ")
    df.to_csv(dest_file_path);

def learn_text(serie):
    vectorizer = TfidfVectorizer(min_df=0.01)
    oued = vectorizer.fit_transform((serie))
    return oued

def learn_category(df, category):
    path = Path(f"{DATA_FRAMES_DIR}/{category}")
    path.mkdir(parents=True, exist_ok=True)

    cur = df[df["root"] == category];

    df_text = cur[["id", "title", "root", "description", "status", "text"]]
    tfidf_title = learn_text(df_text['title'])
    tfidf_description = learn_text(df_text['description'])
    tfidf_text = learn_text(df_text['text'])
    dump(tfidf_title, path / "title_model.sav")
    dump(tfidf_description, path / "description_model.sav")
    dump(tfidf_text, path / "text_model.sav")
    df_text.to_csv(path / "text_preprocess.csv");
    print(f"{category} learned")


def get_roots(df):
    return list(map(int, df["root"].unique()))

#@njit()
def compute_jit(title, description, ids):
    values = (title * TITLE_ALPHA) + (description * (1 - TITLE_ALPHA))
    to_keep = values > THRESHOLD
    values = values[to_keep]
    ids = ids[to_keep]
    srt = np.argsort(values)[::-1]
    values = values[srt]
    ids = ids[srt]
    return (values, ids)
def compute(title, description, ids):
    values = (title * TITLE_ALPHA) + (description * (1 - TITLE_ALPHA))
    torem = values <= THRESHOLD
    values = np.delete(values, torem)
    ids = np.delete(ids, torem)
    srt = np.argsort(values)[::-1]
    values = values[srt]
    ids = ids[srt]
    return (values, ids)
    
    

def learn(src_db):
    df = pd.read_csv(src_db)
    df["title"] = df["title"].fillna(" ")
    df["description"] = df["description"].fillna(" ")
    df["text"] = df["text"].fillna(" ")
    folders = get_roots(df)
    
    for folder in folders:
        learn_category(df, folder)
        print(f"{folder} learned")

#all("../ann.csv", "./result.csv")

#cnt = oued_main.Controller(roots)
#ls = cnt.recommend(239, 946056)


SyntaxError: invalid syntax (265513264.py, line 54)

In [45]:
class CategoryController:
    def __init__(self, df, title, description):
        self.description = description[(df["status"] == 1)|(df["status"] == 4)]
        self.title = title[(df["status"] == 1)|(df["status"] == 4)]
        cur = df[(df["status"] == 1)|(df["status"] == 4)]
        cur["idx"] = range(cur.shape[0])
        cur = cur[["id", "idx"]]
        self.ids = cur["idx"]
        self.ids.index = cur["id"]
        self.tmp = np.array(self.ids.index)

    @Timer(name="get_score")
    def get_score(self, ann_id: int):
        index:int = self.ids.get(ann_id, default=-1)
        if index < 0:
            return []
        
        title_values = cosine_similarity(self.title, self.title[index]).flatten()
        description_values = cosine_similarity(self.description, self.description[index]).flatten()
        ans = compute(title_values, description_values, self.tmp)
        return ans
        #it= filter(lambda p: p[0] > THRESHOLD, compute(title_values, description_values, self.tmp))
        #return sorted(it, key=lambda p: p[0], reverse=True)

class Controller:
    def __init__(self, roots):
        self.roots = roots;
        self.M = dict()
        for root in self.roots:
            path = Path(f"{DATA_FRAMES_DIR}/{root}")
            df = pd.read_csv(path / "text_preprocess.csv")
            title = load(path / "title_model.sav")
            description = load(path / "description_model.sav")
            self.M[root] = CategoryController(df, title, description)

    def recommend(self, root, ann_id):
        if root not in self.M:
            return []
        return self.M[root].get_score(ann_id)


In [46]:
roots = get_roots(df)

In [47]:
roots = [239]
cnt = Controller(roots)

/tmp/ipykernel_52574/2565431058.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur["idx"] = range(cur.shape[0])


In [60]:
%time ans = cnt.recommend(239, 946056)


Elapsed time: 0.0285 seconds
CPU times: user 28.3 ms, sys: 290 µs, total: 28.6 ms
Wall time: 28.8 ms


In [49]:
ans[0].shape
ans[1].shape

(20879,)